# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
# llm_config = {"model":"gpt-4.1"}


GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm_config = {"model":"gemini-1.5-flash", 
            "api_type": "google", 
            "api_key": GOOGLE_API_KEY}


## The task!

In [2]:
# task = '''
#         Write a concise but engaging blogpost about 
#         AI and the Future of Jobs . Make sure the blogpost is
#         within 100 words.
#         '''


task = '''
        Write a concise but engaging blogpost about 
        AI Agent in Finance . Make sure the blogpost is
        within 500 words.
        '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

{'content': "## AI Agents: Revolutionizing the Finance Industry\n\nThe finance industry, long a bastion of meticulous human oversight, is undergoing a dramatic transformation thanks to the rise of AI agents. These aren't just sophisticated algorithms; they're intelligent systems capable of learning, adapting, and making decisions with minimal human intervention.  This shift promises to boost efficiency, reduce risk, and unlock entirely new opportunities within the sector.\n\nOne of the most significant impacts is in **fraud detection**. AI agents can analyze vast datasets of transactions, identifying subtle patterns and anomalies that would be impossible for humans to spot. This proactive approach drastically reduces losses due to fraudulent activities.  Beyond fraud, AI agents excel at **risk management**.  By processing market data and economic indicators in real-time, they can predict potential risks and suggest strategies for mitigation, helping institutions navigate volatile marke

## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

![AI agents Reflection](https://cdn.analyticsvidhya.com/wp-content/uploads/2024/10/image-1-1-1.webp)

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=4,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about 
        AI Agent in Finance . Make sure the blogpost is
        within 500 words.
        

--------------------------------------------------------------------------------
Writer (to Critic):

## AI Agents: Revolutionizing the Financial World

Artificial intelligence (AI) is rapidly transforming numerous sectors, and finance is no exception.  AI agents, sophisticated software programs capable of acting autonomously, are emerging as powerful tools, streamlining processes and offering unprecedented opportunities for growth and efficiency.  But what exactly are AI agents in finance, and how are they changing the game?

AI agents leverage machine learning algorithms to analyze vast datasets, identify patterns, and make decisions with minimal human intervention.  Unlike traditional rule-based systems, AI agents can adapt and learn from new data, improving their performance over time. This adaptability is crucial in 

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 2},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 2},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 2},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 2},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=3,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about 
        AI Agent in Finance . Make sure the blogpost is
        within 500 words.
        

--------------------------------------------------------------------------------
Writer (to Critic):

## AI Agents: Reshaping the Future of Finance

The financial world is undergoing a dramatic transformation, driven by the rapid advancement of artificial intelligence (AI).  No longer a futuristic fantasy, AI agents are becoming integral players, automating processes, improving decision-making, and ultimately, reshaping the financial landscape.

These intelligent systems aren't just glorified calculators.  AI agents leverage machine learning algorithms to analyze vast datasets, identifying patterns and insights invisible to the human eye. This capability translates into significant advantages across various financial sectors.

**Enhanced Fraud Detection:**  AI agents can process millions of transactions in seconds, flaggi

## Get the summary

In [15]:
print(res.summary)

##  AI Agents in Finance: Navigating a Revolution Responsibly

Artificial intelligence (AI) agents are rapidly transforming the financial industry, automating processes, improving decision-making, and creating new opportunities. However, this technological revolution necessitates a responsible approach, prioritizing ethical considerations and strict legal compliance alongside innovation. This requires transparency, accountability, and a commitment to mitigating potential risks.

**Enhanced Fraud Detection and Security:** AI algorithms excel at identifying fraudulent transactions by analyzing vast datasets and detecting subtle patterns imperceptible to human analysts. Studies by firms like McKinsey & Company (McKinsey Global Institute, "Harnessing Automation for a Future that Works," 2017) estimate that AI can improve fraud detection rates by X% (insert specific data from cited source).  This proactive approach significantly reduces financial losses and strengthens security for both ins

In [16]:

print(res.cost)

{'usage_including_cached_inference': {'total_cost': 0.004895549999999999, 'gemini-1.5-flash': {'cost': 0.004895549999999999, 'prompt_tokens': 24870, 'completion_tokens': 10101, 'total_tokens': 34971}}, 'usage_excluding_cached_inference': {'total_cost': 0.004895549999999999, 'gemini-1.5-flash': {'cost': 0.004895549999999999, 'prompt_tokens': 24870, 'completion_tokens': 10101, 'total_tokens': 34971}}}
